# 📊 Analisi Interattiva di Commenti Excel con AI Multi-Provider

Questo notebook permette di analizzare e etichettare automaticamente commenti da file Excel utilizzando diversi provider di intelligenza artificiale.

## 🤖 Provider AI Supportati:
- **🏠 Ollama** - Modelli locali gratuiti (Mixtral, LLaMA, Qwen, etc.)
- **🌐 OpenRouter** - Accesso a modelli multipli via API (molti gratuiti)

## 🎯 Funzionalità:
- **Selezione interattiva** del provider AI
- **Caricamento interattivo** di file Excel
- **Template di prompt** per diversi tipi di analisi
- **Prompt personalizzabile** per l'analisi
- **Etichettatura automatica** dei commenti
- **Report finale** in formato Excel

## 📝 Come usare:
1. Esegui le celle in ordine
2. Scegli il provider AI che preferisci
3. Seleziona il file Excel da analizzare
4. Scegli un template di prompt o creane uno personalizzato
5. Avvia l'analisi e ottieni i risultati

## 💡 Vantaggi Multi-Provider:
- **Ollama**: Completamente gratuito, privacy totale
- **OpenRouter**: Accesso a modelli all'avanguardia, molti gratuiti

## 📚 1. Importazione Librerie

Importiamo tutte le librerie necessarie per l'analisi.

In [13]:
import pandas as pd
import logging
import os
import time
from datetime import datetime
from langchain_ollama import OllamaLLM
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import json

# Carica le variabili di ambiente dal file .env
load_dotenv()

# Configurazione del logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("operazioni_log.log"),
        logging.StreamHandler()
    ]
)

print("✅ Librerie importate con successo!")

✅ Librerie importate con successo!


## 🔧 2. Configurazione Provider AI

Configuriamo la connessione con il provider AI scelto (OpenRouter o Ollama).

In [14]:
# Configurazione provider AI multipli
AI_PROVIDER = os.getenv('AI_PROVIDER', 'ollama').lower()
TEMPERATURE = float(os.getenv('TEMPERATURE', '0.7'))

class OpenRouterLLM:
    """Wrapper personalizzato per OpenRouter senza dipendere da OpenAI"""
    
    def __init__(self, model, api_key, temperature=0.7):
        self.model = model
        self.api_key = api_key
        self.temperature = temperature
        self.base_url = "https://openrouter.ai/api/v1/chat/completions"
    
    def invoke(self, messages):
        """Invoca il modello OpenRouter con i messaggi forniti"""
        
        # Se messages è una stringa, convertila in formato chat
        if isinstance(messages, str):
            formatted_messages = [{"role": "user", "content": messages}]
        else:
            formatted_messages = messages
            
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "HTTP-Referer": "http://localhost:8888",
            "X-Title": "Analisi Excel AI",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": formatted_messages,
            "temperature": self.temperature
        }
        
        try:
            response = requests.post(self.base_url, headers=headers, json=data, timeout=60)
            response.raise_for_status()
            
            result = response.json()
            return result['choices'][0]['message']['content']
            
        except requests.exceptions.RequestException as e:
            raise ValueError(f"❌ Errore nella chiamata a OpenRouter: {e}")
        except (KeyError, IndexError) as e:
            raise ValueError(f"❌ Formato risposta non valido da OpenRouter: {e}")

def check_all_providers():
    """Verifica lo stato di tutti i provider disponibili"""
    print("🔍 VERIFICA STATO PROVIDER AI")
    print("=" * 50)
    
    # Verifica Ollama
    print("\n🏠 OLLAMA (Locale)")
    ollama_status = "❌"
    ollama_info = ""
    try:
        base_url = os.getenv('OLLAMA_BASE_URL', 'http://192.168.129.14:11435')
        model = os.getenv('OLLAMA_MODEL', 'mixtral:8x7b')
        
        response = requests.get(f"{base_url}/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json().get('models', [])
            ollama_status = "✅"
            ollama_info = f"Server: {base_url} | Modello: {model} | Disponibili: {len(models)}"
        else:
            ollama_info = f"Server risponde con errore: {response.status_code}"
    except Exception as e:
        ollama_info = f"Server non raggiungibile: {base_url}"
    
    print(f"{ollama_status} Ollama: {ollama_info}")
    
    # Verifica OpenRouter
    print("\n🌐 OPENROUTER (API)")
    openrouter_status = "❌"
    openrouter_info = ""
    try:
        api_key = os.getenv('OPENROUTER_API_KEY', '')
        model = os.getenv('OPENROUTER_MODEL', 'nvidia/llama-3.1-nemotron-ultra-253b-v1')
        
        if api_key and api_key != 'sk-or-your-openrouter-api-key-here':
            openrouter_status = "✅"
            openrouter_info = f"API Key: {api_key[:15]}...{api_key[-4:]} | Modello: {model}"
        else:
            openrouter_info = "API Key non configurata"
    except Exception as e:
        openrouter_info = f"Errore configurazione: {e}"
    
    print(f"{openrouter_status} OpenRouter: {openrouter_info}")
    
    print("\n" + "=" * 50)
    return ollama_status == "✅", openrouter_status == "✅"

def create_llm():
    """Crea l'istanza LLM basata sul provider configurato"""
    
    if AI_PROVIDER == 'openrouter':
        api_key = os.getenv('OPENROUTER_API_KEY')
        model = os.getenv('OPENROUTER_MODEL', 'nvidia/llama-3.1-nemotron-ultra-253b-v1')
        
        if not api_key:
            raise ValueError("❌ OPENROUTER_API_KEY non trovata nel file .env")
        
        llm = OpenRouterLLM(
            model=model,
            api_key=api_key,
            temperature=TEMPERATURE
        )
        return llm
        
    elif AI_PROVIDER == 'ollama':
        base_url = os.getenv('OLLAMA_BASE_URL', 'http://192.168.129.14:11435')
        model = os.getenv('OLLAMA_MODEL', 'mixtral:8x7b')
        
        # Verifica che Ollama sia raggiungibile
        try:
            response = requests.get(f"{base_url}/api/tags", timeout=5)
            if response.status_code != 200:
                raise ValueError(f"❌ Ollama non raggiungibile su {base_url}")
        except requests.exceptions.RequestException:
            raise ValueError(f"❌ Impossibile connettersi a Ollama su {base_url}")
        
        llm = OllamaLLM(
            model=model,
            base_url=base_url,
            temperature=TEMPERATURE
        )
        return llm
        
    else:
        raise ValueError(f"❌ Provider AI non supportato: {AI_PROVIDER}. Usa: openrouter, ollama")

# Verifica tutti i provider
ollama_ok, openrouter_ok = check_all_providers()

# Crea l'istanza LLM per il provider attivo
try:
    llm = create_llm()
    print(f"\n🎯 PROVIDER ATTIVO: {AI_PROVIDER.upper()} ✅")
except Exception as e:
    print(f"\n❌ ERRORE PROVIDER ATTIVO ({AI_PROVIDER.upper()}): {e}")
    llm = None

# Riepilogo finale
print(f"\n📊 RIEPILOGO CONFIGURAZIONE:")
print(f"   🏠 Ollama: {'✅ Disponibile' if ollama_ok else '❌ Non disponibile'}")
print(f"   🌐 OpenRouter: {'✅ Configurato' if openrouter_ok else '❌ Non configurato'}")
print(f"   🎯 Attivo: {AI_PROVIDER.upper()} {'✅' if llm else '❌'}")

if not ollama_ok and not openrouter_ok:
    print("\n⚠️  ATTENZIONE: Nessun provider configurato correttamente!")
    print("   Configura almeno uno dei provider per utilizzare il notebook.")

🔍 VERIFICA STATO PROVIDER AI

🏠 OLLAMA (Locale)
✅ Ollama: Server: http://192.168.129.14:11435 | Modello: mixtral:8x7b | Disponibili: 12

🌐 OPENROUTER (API)
✅ OpenRouter: API Key: sk-or-v1-872377...f823 | Modello: nvidia/llama-3.1-nemotron-ultra-253b-v1


🎯 PROVIDER ATTIVO: OLLAMA ✅

📊 RIEPILOGO CONFIGURAZIONE:
   🏠 Ollama: ✅ Disponibile
   🌐 OpenRouter: ✅ Configurato
   🎯 Attivo: OLLAMA ✅


## 🎛️ 2.1. Selezione Provider AI

Scegli il provider AI che vuoi utilizzare per l'analisi. Puoi cambiarlo in qualsiasi momento.

In [ ]:
# 🎛️ SELEZIONE PROVIDER AI INTERATTIVA

import ipywidgets as widgets
from IPython.display import display, clear_output

# Widget per selezionare il provider
provider_dropdown = widgets.Dropdown(
    options=[
        ('🏠 Ollama (Locale/Gratuito) - mixtral:8x7b', 'ollama'),
        ('🌐 OpenRouter (API/Gratuito) - Nemotron Ultra 253B', 'openrouter')
    ],
    value=os.getenv('AI_PROVIDER', 'ollama'),
    description='Provider:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Widget per mostrare i modelli disponibili
model_info = widgets.HTML(
    value="<p><b>Modelli disponibili verranno mostrati qui...</b></p>"
)

# Widget per selezionare il modello (specifico per provider)
model_dropdown = widgets.Dropdown(
    options=[],
    description='Modello:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Output area per feedback
provider_output = widgets.Output()

def update_models(provider):
    """Aggiorna i modelli disponibili in base al provider selezionato"""
    
    if provider == 'ollama':
        # Modelli Ollama disponibili
        ollama_models = [
            ('mixtral:8x7b (26GB) - Molto potente ⭐ CONSIGLIATO', 'mixtral:8x7b'),
            ('deepseek-r1:latest (4.7GB) - Ragionamento avanzato', 'deepseek-r1:latest'),
            ('qwen2.5:7b (4.7GB) - Equilibrio qualità/velocità', 'qwen2.5:7b'),
            ('qwen3:32b (20GB) - Modello grande e capace', 'qwen3:32b'),
            ('llama4:16x17b (67GB) - Massima qualità', 'llama4:16x17b'),
            ('phi4:14b (9.1GB) - Microsoft, buone prestazioni', 'phi4:14b'),
            ('llama3:latest (4.7GB) - Affidabile e testato', 'llama3:latest'),
            ('qwen:110b (62GB) - Modello gigante', 'qwen:110b')
        ]
        model_dropdown.options = ollama_models
        model_dropdown.value = os.getenv('OLLAMA_MODEL', 'mixtral:8x7b')
        
        model_info.value = """
        <div style='background: #e8f4f8; padding: 10px; border-radius: 5px; margin: 10px 0;'>
        <h4>🏠 Ollama (Locale)</h4>
        <p><b>Vantaggi:</b> Completamente gratuito, privacy totale, nessun limite di utilizzo</p>
        <p><b>Requisiti:</b> Server Ollama attivo su http://192.168.129.14:11435</p>
        <p><b>Prestazioni:</b> Dipendono dall'hardware locale</p>
        </div>
        """
        
    elif provider == 'openrouter':
        # Modelli OpenRouter disponibili (gratuiti)
        openrouter_models = [
            ('nvidia/llama-3.1-nemotron-ultra-253b-v1 - Reasoning 253B ⭐', 'nvidia/llama-3.1-nemotron-ultra-253b-v1'),
            ('deepseek/deepseek-r1-distill-qwen-14b - Reasoning 14B', 'deepseek/deepseek-r1-distill-qwen-14b'),
            ('arliai/qwq-32b-arliai-rpr-v1 - Creative Writing 32B', 'arliai/qwq-32b-arliai-rpr-v1'),
            ('mistralai/mistral-small-3 - Fast 24B', 'mistralai/mistral-small-3'),
            ('reka/reka-flash-3 - General Purpose 21B', 'reka/reka-flash-3'),
            ('featherless/qwerky-72b - Linear Attention 72B', 'featherless/qwerky-72b'),
            ('sarvamai/sarvam-m - Multilingual 24B', 'sarvamai/sarvam-m'),
            ('google/gemma-3-4b - Multimodal 4B', 'google/gemma-3-4b')
        ]
        model_dropdown.options = openrouter_models
        model_dropdown.value = os.getenv('OPENROUTER_MODEL', 'nvidia/llama-3.1-nemotron-ultra-253b-v1')
        
        model_info.value = """
        <div style='background: #f0f8e8; padding: 10px; border-radius: 5px; margin: 10px 0;'>
        <h4>🌐 OpenRouter (API)</h4>
        <p><b>Vantaggi:</b> Accesso a modelli all'avanguardia, molti modelli gratuiti</p>
        <p><b>Requisiti:</b> API Key OpenRouter (gratuita)</p>
        <p><b>Costi:</b> Modelli selezionati sono GRATUITI</p>
        </div>
        """

def on_provider_change(change):
    """Callback quando cambia il provider"""
    with provider_output:
        clear_output()
        new_provider = change['new']
        update_models(new_provider)
        print(f"✅ Provider selezionato: {new_provider.upper()}")

def on_model_change(change):
    """Callback quando cambia il modello"""
    with provider_output:
        clear_output()
        print(f"✅ Modello selezionato: {change['new']}")

# Collega i callback
provider_dropdown.observe(on_provider_change, names='value')
model_dropdown.observe(on_model_change, names='value')

# Inizializza i modelli per il provider corrente
update_models(provider_dropdown.value)

# Pulsante per applicare la configurazione
apply_config_button = widgets.Button(
    description='🔧 Applica Configurazione',
    button_style='success',
    layout=widgets.Layout(width='250px')
)

def on_apply_config(b):
    """Applica la configurazione selezionata"""
    with provider_output:
        clear_output()
        
        selected_provider = provider_dropdown.value
        selected_model = model_dropdown.value
        
        # Aggiorna le variabili di ambiente temporaneamente
        os.environ['AI_PROVIDER'] = selected_provider
        
        if selected_provider == 'ollama':
            os.environ['OLLAMA_MODEL'] = selected_model
        elif selected_provider == 'openrouter':
            os.environ['OPENROUTER_MODEL'] = selected_model
        
        # Aggiorna anche le variabili globali per applicare immediatamente
        global AI_PROVIDER, llm
        AI_PROVIDER = selected_provider
        
        # Ricrea l'istanza LLM con la nuova configurazione
        try:
            llm = create_llm()
            print(f"🎯 Configurazione applicata e attivata:")
            print(f"   Provider: {selected_provider.upper()} ✅")
            print(f"   Modello: {selected_model}")
            print(f"   Stato: Pronto per l'analisi ✅")
        except Exception as e:
            print(f"❌ Errore nell'applicazione della configurazione: {e}")
            print(f"   Provider: {selected_provider.upper()}")
            print(f"   Modello: {selected_model}")
            print("   Verifica la configurazione del provider selezionato.")

apply_config_button.on_click(on_apply_config)

# Mostra l'interfaccia
display(widgets.VBox([
    widgets.HTML("<h3>🎛️ Seleziona Provider e Modello AI</h3>"),
    provider_dropdown,
    model_info,
    model_dropdown,
    apply_config_button,
    provider_output
]))

## 🛠️ 3. Funzioni di Supporto

Definiamo le funzioni necessarie per l'analisi (basate su generico.py).

In [ ]:
# 📁 Funzione per creare la cartella di output
def create_output_dir(tipo_analisi):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f'{tipo_analisi}_output_{timestamp}'
    os.makedirs(output_dir, exist_ok=True)
    logging.info(f"Cartella di output creata: {output_dir}")
    return output_dir

# 📄 Funzione per leggere il file Excel
def load_excel(file_path, colonna_riferimento):
    try:
        df = pd.read_excel(file_path)
        logging.info(f"File Excel '{file_path}' caricato con successo.")
        return df, colonna_riferimento
    except Exception as e:
        logging.error(f"Errore durante il caricamento del file Excel: {e}")
        raise

# 🔍 FASE 1: Analisi globale della colonna per generare etichette dinamiche
def analizza_colonna_completa(df, colonna_riferimento, prompt_personalizzato, tipo_analisi):
    """
    Analizza l'intera colonna per identificare pattern e generare etichette dinamiche
    """
    logging.info(f"Inizio analisi globale della colonna '{colonna_riferimento}'")
    
    # Estrae tutti i commenti non vuoti
    commenti_validi = df[colonna_riferimento].dropna().astype(str).tolist()
    
    if not commenti_validi:
        raise ValueError("❌ Nessun commento valido trovato nella colonna")
    
    # Combina tutti i commenti per l'analisi globale
    testo_completo = "\n".join(commenti_validi)
    
    # Prompt specifico per generare etichette dinamiche
    prompt_etichette = f"""Analizza TUTTO il seguente testo che contiene {len(commenti_validi)} commenti di un progetto scolastico.

OBIETTIVO: Genera un dizionario completo di etichette dinamiche basate sui contenuti REALI di questi commenti.

ISTRUZIONI:
1. Leggi tutti i commenti per identificare temi, pattern e concetti ricorrenti
2. Crea etichette specifiche e pertinenti al contenuto effettivo
3. Ogni etichetta deve rappresentare un concetto distintivo presente nei dati
4. Fornisci una descrizione dettagliata per ogni etichetta
5. Includi anche etichette per concetti minoritari ma significativi

FORMATO RICHIESTO:
Per ogni etichetta, scrivi:
ETICHETTA: [nome_breve]
DESCRIZIONE: [spiegazione dettagliata di cosa rappresenta]
ESEMPI: [parole chiave o frasi tipiche]
---

TEMPLATE ANALISI:
{prompt_personalizzato}

TESTO DA ANALIZZARE:
{testo_completo}

Genera massimo 25 etichette dinamiche basate sui contenuti reali."""

    try:
        if AI_PROVIDER == 'ollama':
            risposta_etichette = llm.invoke(prompt_etichette)
        else:
            messages = [
                {"role": "system", "content": f"Sei un esperto analista che genera etichette dinamiche per {tipo_analisi} di progetti educativi."},
                {"role": "user", "content": prompt_etichette}
            ]
            risposta_etichette = llm.invoke(messages)
        
        # Parsing delle etichette dalla risposta
        etichette_dinamiche = parse_etichette_dinamiche(str(risposta_etichette))
        
        logging.info(f"Analisi globale completata: {len(etichette_dinamiche)} etichette generate")
        
        return {
            "etichette_dinamiche": etichette_dinamiche,
            "statistiche_colonna": {
                "totale_commenti": len(commenti_validi),
                "lunghezza_media": sum(len(c) for c in commenti_validi) / len(commenti_validi),
                "commenti_vuoti": len(df) - len(commenti_validi)
            },
            "analisi_completa": risposta_etichette
        }
        
    except Exception as e:
        logging.error(f"Errore durante l'analisi globale: {e}")
        raise

# 🏷️ FASE 2: Etichettatura avanzata con coefficienti di confidenza
def etichetta_con_coefficiente(df, etichette_dinamiche, colonna_riferimento, tipo_analisi, soglia_confidenza=0.3):
    """
    Etichetta ogni cella con coefficienti di corrispondenza per tutte le etichette
    """
    global fase_label, progress_bar  # Accesso ai widget di progresso
    
    logging.info(f"Inizio etichettatura con coefficienti (soglia: {soglia_confidenza})")
    
    risultati = {
        "etichette_principali": [],
        "coefficienti_principali": [],
        "etichette_secondarie": [],
        "coefficienti_completi": [],
        "confidenza_media": []
    }
    
    # Prepara la lista delle etichette per il prompt
    lista_etichette = "\n".join([
        f"- {nome}: {info['descrizione']}" 
        for nome, info in etichette_dinamiche.items()
    ])
    
    for index, row in df.iterrows():
        commento = row[colonna_riferimento]
        
        if pd.isna(commento):
            risultati["etichette_principali"].append("Vuota")
            risultati["coefficienti_principali"].append(0.0)
            risultati["etichette_secondarie"].append("")
            risultati["coefficienti_completi"].append("{}")
            risultati["confidenza_media"].append(0.0)
            continue
        
        # Calcola progresso dettagliato
        numero_corrente = index + 1
        totale_commenti = len(df)
        percentuale = int((numero_corrente / totale_commenti) * 100)
        
        # Aggiorna l'indicatore di progresso con formato p% 4/100
        fase_label.value = f"<b>🏷️ FASE 2: Etichettatura {percentuale}% {numero_corrente}/{totale_commenti}</b>"
        
        # Aggiorna anche la progress bar (40-80% è riservato per la fase 2)
        progress_fase2 = 40 + int((numero_corrente / totale_commenti) * 40)
        progress_bar.value = progress_fase2
        
        logging.info(f"Analizzando commento {numero_corrente}/{totale_commenti} ({percentuale}%)")
        
        # Prompt per calcolare coefficienti per tutte le etichette
        prompt_coefficienti = f"""Analizza questo commento e calcola quanto si adatta a OGNI etichetta (coefficiente 0.0-1.0).

ETICHETTE DISPONIBILI:
{lista_etichette}

COMMENTO DA ANALIZZARE:
"{commento}"

ISTRUZIONI:
1. Per OGNI etichetta, assegna un coefficiente da 0.0 (nessuna corrispondenza) a 1.0 (perfetta corrispondenza)
2. Sii preciso nella valutazione - usa l'intera scala 0.0-1.0
3. Identifica l'etichetta principale (coefficiente più alto)
4. Indica anche etichette secondarie significative (sopra {soglia_confidenza})

FORMATO RISPOSTA:
PRINCIPALE: [nome_etichetta] (coefficiente: 0.XX)
SECONDARIE: [etichetta1] (0.XX), [etichetta2] (0.XX)
TUTTI_COEFFICIENTI: {{\"etichetta1\": 0.XX, \"etichetta2\": 0.XX, ...}}
CONFIDENZA_GENERALE: 0.XX"""

        try:
            if AI_PROVIDER == 'ollama':
                risposta = llm.invoke(prompt_coefficienti)
            else:
                messages = [
                    {"role": "system", "content": f"Sei un analista esperto che calcola coefficienti di corrispondenza per {tipo_analisi}."},
                    {"role": "user", "content": prompt_coefficienti}
                ]
                risposta = llm.invoke(messages)
            
            # Parsing della risposta per estrarre coefficienti
            risultato_parsing = parse_coefficienti_risposta(str(risposta), soglia_confidenza)
            
            risultati["etichette_principali"].append(risultato_parsing["principale"])
            risultati["coefficienti_principali"].append(risultato_parsing["coeff_principale"])
            risultati["etichette_secondarie"].append(risultato_parsing["secondarie"])
            risultati["coefficienti_completi"].append(risultato_parsing["tutti_coefficienti"])
            risultati["confidenza_media"].append(risultato_parsing["confidenza_generale"])
            
            logging.info(f"[{percentuale}% {numero_corrente}/{totale_commenti}] Etichetta: {risultato_parsing['principale']} ({risultato_parsing['coeff_principale']:.2f})")
            
            # Mostra progresso ogni 10 commenti o su milestone importanti
            if numero_corrente % 10 == 0 or numero_corrente in [1, 5] or numero_corrente == totale_commenti:
                print(f"   📊 Progresso: {percentuale}% ({numero_corrente}/{totale_commenti} commenti completati)")
            
        except Exception as e:
            logging.error(f"[{percentuale}% {numero_corrente}/{totale_commenti}] Errore nell'etichettatura: {e}")
            risultati["etichette_principali"].append("Errore")
            risultati["coefficienti_principali"].append(0.0)
            risultati["etichette_secondarie"].append("")
            risultati["coefficienti_completi"].append("{}")
            risultati["confidenza_media"].append(0.0)
        
        # Pausa per evitare rate limiting
        time.sleep(1)
    
    # Calcola statistiche finali
    coefficienti_validi = [c for c in risultati["coefficienti_principali"] if c > 0]
    risultati["statistiche_confidenza"] = {
        "media_coefficienti": sum(coefficienti_validi) / len(coefficienti_validi) if coefficienti_validi else 0,
        "alta_confidenza": len([c for c in coefficienti_validi if c >= 0.7]),
        "media_confidenza": len([c for c in coefficienti_validi if 0.4 <= c < 0.7]),
        "bassa_confidenza": len([c for c in coefficienti_validi if 0.1 <= c < 0.4]),
        "molto_bassa": len([c for c in coefficienti_validi if c < 0.1])
    }
    
    logging.info("Etichettatura con coefficienti completata")
    return risultati

# 🔧 Funzioni di supporto per parsing
def parse_etichette_dinamiche(risposta_ai):
    """Estrae etichette e descrizioni dalla risposta dell'AI"""
    etichette = {}
    linee = risposta_ai.split('\n')
    
    etichetta_corrente = None
    descrizione_corrente = ""
    esempi_correnti = ""
    
    for linea in linee:
        linea = linea.strip()
        if linea.startswith('ETICHETTA:'):
            if etichetta_corrente:
                etichette[etichetta_corrente] = {
                    'descrizione': descrizione_corrente,
                    'esempi': esempi_correnti
                }
            etichetta_corrente = linea.replace('ETICHETTA:', '').strip()
            descrizione_corrente = ""
            esempi_correnti = ""
        elif linea.startswith('DESCRIZIONE:'):
            descrizione_corrente = linea.replace('DESCRIZIONE:', '').strip()
        elif linea.startswith('ESEMPI:'):
            esempi_correnti = linea.replace('ESEMPI:', '').strip()
    
    # Aggiungi l'ultima etichetta
    if etichetta_corrente:
        etichette[etichetta_corrente] = {
            'descrizione': descrizione_corrente,
            'esempi': esempi_correnti
        }
    
    return etichette

def parse_coefficienti_risposta(risposta, soglia):
    """Estrae coefficienti dalla risposta dell'AI"""
    import re
    
    risultato = {
        "principale": "Incerto",
        "coeff_principale": 0.0,
        "secondarie": "",
        "tutti_coefficienti": "{}",
        "confidenza_generale": 0.0
    }
    
    # Estrae etichetta principale
    match_principale = re.search(r'PRINCIPALE:\s*([^(]+)\s*\(coefficiente:\s*([\d.]+)\)', risposta)
    if match_principale:
        risultato["principale"] = match_principale.group(1).strip()
        risultato["coeff_principale"] = float(match_principale.group(2))
    
    # Estrae etichette secondarie
    match_secondarie = re.search(r'SECONDARIE:\s*(.+)', risposta)
    if match_secondarie:
        risultato["secondarie"] = match_secondarie.group(1).strip()
    
    # Estrae coefficienti completi
    match_coefficienti = re.search(r'TUTTI_COEFFICIENTI:\s*(\{.+\})', risposta)
    if match_coefficienti:
        risultato["tutti_coefficienti"] = match_coefficienti.group(1)
    
    # Estrae confidenza generale
    match_confidenza = re.search(r'CONFIDENZA_GENERALE:\s*([\d.]+)', risposta)
    if match_confidenza:
        risultato["confidenza_generale"] = float(match_confidenza.group(1))
    
    return risultato

# 💾 Salvataggio risultati avanzati
def salva_risultati_avanzati(df, risultati_etichettatura, etichette_dinamiche, output_dir, tipo_analisi):
    """Salva risultati con colonne aggiuntive per coefficienti"""
    
    # Crea DataFrame avanzato
    df_risultati = df.copy()
    df_risultati['Etichetta_Principale'] = risultati_etichettatura["etichette_principali"]
    df_risultati['Coefficiente_Principale'] = risultati_etichettatura["coefficienti_principali"]
    df_risultati['Etichette_Secondarie'] = risultati_etichettatura["etichette_secondarie"]
    df_risultati['Confidenza_Generale'] = risultati_etichettatura["confidenza_media"]
    df_risultati['Coefficienti_Completi'] = risultati_etichettatura["coefficienti_completi"]
    
    # Salva Excel avanzato
    excel_path = os.path.join(output_dir, f'report_avanzato_{tipo_analisi}.xlsx')
    df_risultati.to_excel(excel_path, index=False)
    
    # Salva dizionario etichette
    etichette_path = os.path.join(output_dir, f'etichette_dinamiche_{tipo_analisi}.json')
    with open(etichette_path, 'w', encoding='utf-8') as f:
        json.dump(etichette_dinamiche, f, ensure_ascii=False, indent=2)
    
    # Salva statistiche
    statistiche_path = os.path.join(output_dir, f'statistiche_confidenza_{tipo_analisi}.json')
    with open(statistiche_path, 'w', encoding='utf-8') as f:
        json.dump(risultati_etichettatura["statistiche_confidenza"], f, ensure_ascii=False, indent=2)
    
    logging.info(f"Risultati avanzati salvati in {output_dir}")
    return excel_path, etichette_path, statistiche_path

# 📝 Funzione per aggiornare automaticamente il prompt in base al tipo di analisi
def aggiorna_prompt_automatico():
    """Aggiorna il prompt in base al tipo di analisi selezionato"""
    if 'tipo_analisi' in globals() and 'template_prompts' in globals() and tipo_analisi in template_prompts:
        global prompt_personalizzato
        prompt_personalizzato = template_prompts[tipo_analisi]['prompt']
        print(f"✅ Prompt automatico caricato per: {template_prompts[tipo_analisi]['nome']}")
        return True
    return False

print("✅ Sistema avanzato di analisi e etichettatura definito!")

✅ Sistema avanzato di analisi e etichettatura definito!


In [17]:
# 🎯 DEFINIZIONE TEMPLATE DI PROMPT PER ANALISI QUALITATIVE

template_prompts = {
    "sentiment_analysis": {
        "nome": "📊 Sentiment Analysis",
        "descrizione": "Analizza il sentiment (positivo, negativo, neutro) dei commenti",
        "prompt": """Analizza il sentiment e le emozioni espressi nei seguenti commenti.

Per ogni commento, identifica:
1. **Sentiment generale**: Positivo, Negativo, Neutro, Misto
2. **Intensità emotiva**: Bassa, Media, Alta
3. **Emozioni specifiche**: Gioia, Rabbia, Tristezza, Paura, Sorpresa, Disgusto, Anticipazione, Fiducia
4. **Indicatori linguistici**: Parole chiave che indicano il sentiment

Crea categorie di sentiment che raggruppino i commenti simili. Massimo 15 categorie.
Per ogni categoria, fornisci:
- Nome della categoria (es: "Entusiasmo-Alto", "Preoccupazione-Moderata")
- Descrizione del sentiment
- Esempi di parole/frasi indicative

Testo da analizzare:"""
    },
    
    "pro_contro": {
        "nome": "⚖️ Analisi Pro e Contro",
        "descrizione": "Identifica vantaggi, svantaggi, benefici e criticità",
        "prompt": """Analizza i pro e i contro emersi dai seguenti commenti relativi al progetto educativo.

Identifica e categorizza:

**ASPETTI POSITIVI (PRO):**

**Benefici Educativi:**
- Miglioramento apprendimento
- Sviluppo competenze specifiche
- Innovazione didattica
- Personalizzazione percorsi
- Risultati accademici

**Benefici Sociali:**
- Inclusione e integrazione
- Collaborazione tra studenti
- Coinvolgimento famiglie
- Apertura verso territorio
- Sviluppo comunità educante

**Benefici Organizzativi:**
- Efficienza processi
- Ottimizzazione risorse
- Flessibilità organizzativa
- Comunicazione migliorata
- Coordinamento attività

**Benefici Emotivi/Motivazionali:**
- Aumento motivazione
- Maggiore soddisfazione
- Senso di appartenenza
- Autostima rafforzata
- Benessere psicologico

**ASPETTI NEGATIVI (CONTRO):**

**Criticità Educative:**
- Difficoltà apprendimento
- Lacune formative
- Metodi inadeguati
- Sovraccarico cognitivo
- Risultati insoddisfacenti

**Criticità Sociali:**
- Conflitti interpersonali
- Esclusione di alcuni gruppi
- Resistenze familiari
- Isolamento sociale
- Disparità di trattamento

**Criticità Organizzative:**
- Carenza risorse
- Problemi logistici
- Tempi inadeguati
- Comunicazione inefficace
- Coordinamento carente

**Criticità Emotivi/Motivazionali:**
- Stress e ansia
- Demotivazione
- Frustrazione
- Bassa autostima
- Malessere generale

**ASPETTI AMBIVALENTI:**
- Elementi che presentano sia vantaggi che svantaggi
- Benefici con costi associati
- Opportunità con rischi

Per ogni categoria identificata:
- Livello di impatto (Alto, Medio, Basso)
- Frequenza di menzione
- Intensità emotiva associata
- Possibili soluzioni (per i contro) o rafforzamenti (per i pro)
- Stakeholder coinvolti

Massimo 20 categorie totali tra pro e contro.

Testo da analizzare:"""
    },
    
    "comportamenti": {
        "nome": "👥 Analisi Comportamentale",
        "descrizione": "Identifica comportamenti, atteggiamenti e modalità di partecipazione",
        "prompt": """Analizza i comportamenti e gli atteggiamenti descritti nei seguenti commenti.

Identifica e categorizza:
1. **Comportamenti osservati**: Partecipazione attiva, passiva, collaborativa, individuale
2. **Modalità di interazione**: Con insegnanti, con pari, con materiali, con ambiente
3. **Atteggiamenti**: Apertura, resistenza, curiosità, disinteresse, entusiasmo
4. **Stili di apprendimento**: Visivo, auditivo, cinestetico, sociale, individuale
5. **Strategie utilizzate**: Problem-solving, memorizzazione, sperimentazione

Per ogni categoria comportamentale:
- Nome breve e descrittivo
- Descrizione del comportamento osservato
- Contesto in cui si manifesta
- Frequenza/intensità

Massimo 18 categorie comportamentali.

Testo da analizzare:"""
    },
    
    "emozioni_dettagliate": {
        "nome": "💝 Analisi Emotiva Dettagliata",
        "descrizione": "Analisi approfondita delle emozioni e stati d'animo",
        "prompt": """Conduci un'analisi emotiva approfondita dei seguenti commenti.

Analizza le emozioni su più livelli:

**EMOZIONI PRIMARIE:**
- Gioia/Felicità - Tristezza/Malinconia
- Rabbia/Frustrazione - Calma/Serenità  
- Paura/Ansia - Sicurezza/Tranquillità
- Sorpresa/Stupore - Anticipazione/Aspettativa

**EMOZIONI SOCIALI:**
- Orgoglio - Vergogna
- Gratitudine - Risentimento
- Empatia - Indifferenza
- Appartenenza - Esclusione

**STATI MOTIVAZIONALI:**
- Entusiasmo - Apatia
- Determinazione - Rassegnazione
- Curiosità - Noia
- Fiducia - Sfiducia

Per ogni categoria emotiva identificata:
- Nome dell'emozione/stato
- Intensità (1-5)
- Durata (momentanea, persistente)
- Trigger scatenanti
- Manifestazioni concrete

Massimo 20 categorie emotive.

Testo da analizzare:"""
    },
    
    "motivazioni": {
        "nome": "🎯 Analisi Motivazionale",
        "descrizione": "Identifica motivazioni, bisogni e driver comportamentali",
        "prompt": """Analizza le motivazioni e i bisogni espressi nei seguenti commenti.

Identifica e categorizza:

**MOTIVAZIONI INTRINSECHE:**
- Curiosità intellettuale
- Desiderio di competenza/maestria
- Bisogno di autonomia
- Piacere dell'apprendimento
- Sfida personale

**MOTIVAZIONI ESTRINSECHE:**
- Riconoscimento sociale
- Voti/valutazioni
- Approvazione genitori/insegnanti
- Competizione con pari
- Ricompense concrete

**BISOGNI FONDAMENTALI:**
- Appartenenza e connessione
- Sicurezza e stabilità
- Stima e riconoscimento
- Autorealizzazione
- Significato e scopo

**OSTACOLI MOTIVAZIONALI:**
- Paura del fallimento
- Mancanza di fiducia
- Sovraccarico cognitivo
- Disallineamento valori

Per ogni categoria motivazionale:
- Nome del driver motivazionale
- Livello di intensità
- Fattori di rinforzo
- Potenziali ostacoli

Massimo 16 categorie.

Testo da analizzare:"""
    },
    
    "bisogni_necessita": {
        "nome": "🔍 Analisi Bisogni e Necessità",
        "descrizione": "Identifica bisogni espressi, impliciti e necessità emergenti",
        "prompt": """Analizza i bisogni e le necessità emergenti dai seguenti commenti.

Classifica i bisogni in:

**BISOGNI EDUCATIVI:**
- Supporto personalizzato
- Materiali adeguati
- Metodologie diverse
- Feedback costruttivo
- Tempo sufficiente

**BISOGNI SOCIO-EMOTIVI:**
- Relazioni positive
- Senso di appartenenza
- Sicurezza emotiva
- Supporto psicologico
- Gestione stress

**BISOGNI ORGANIZZATIVI:**
- Chiarezza istruzioni
- Struttura/routine
- Flessibilità orari
- Spazi adeguati
- Tecnologie funzionanti

**BISOGNI COMUNICATIVI:**
- Ascolto attivo
- Comunicazione chiara
- Feedback frequente
- Canali multipli
- Dialogo bidirezionale

**BISOGNI EMERGENTI:**
- Competenze digitali
- Soft skills
- Orientamento futuro
- Sostenibilità
- Inclusività

Per ogni bisogno identificato:
- Urgenza (bassa, media, alta)
- Complessità di risposta
- Risorse necessarie
- Impatto se non soddisfatto

Massimo 20 categorie di bisogni.

Testo da analizzare:"""
    },
    
    "barriere_ostacoli": {
        "nome": "🚧 Analisi Barriere e Ostacoli",
        "descrizione": "Identifica ostacoli, difficoltà e barriere all'apprendimento",
        "prompt": """Identifica e categorizza gli ostacoli e le barriere presenti nei commenti.

Analizza ostacoli su diversi livelli:

**BARRIERE COGNITIVE:**
- Difficoltà di comprensione
- Sovraccarico informativo
- Lacune prerequisiti
- Stili apprendimento non supportati
- Problemi attenzione/concentrazione

**BARRIERE EMOTIVE:**
- Ansia da prestazione
- Bassa autostima
- Paura del giudizio
- Demotivazione
- Stress emotivo

**BARRIERE SOCIALI:**
- Isolamento sociale
- Conflitti interpersonali
- Differenze culturali
- Problemi comunicazione
- Esclusione dal gruppo

**BARRIERE STRUTTURALI:**
- Risorse inadeguate
- Tempo insufficiente
- Spazi non idonei
- Tecnologie obsolete
- Organizzazione carente

**BARRIERE SISTEMICHE:**
- Politiche rigide
- Mancanza formazione docenti
- Valutazione inadeguata
- Curriculum non aggiornato
- Burocrazia eccessiva

Per ogni barriera:
- Gravità dell'impatto (1-5)
- Frequenza di manifestazione
- Possibili soluzioni
- Livello di intervento richiesto

Massimo 18 categorie di barriere.

Testo da analizzare:"""
    },
    
    "relazioni_dinamiche": {
        "nome": "🤝 Analisi Relazionale e Dinamiche",
        "descrizione": "Analizza relazioni interpersonali e dinamiche di gruppo",
        "prompt": """Analizza le relazioni e le dinamiche interpersonali descritte nei commenti.

Esamina:

**RELAZIONI INTERPERSONALI:**
- Studente-Insegnante: Collaborativa, Conflittuale, Distante, Supportiva
- Studente-Studente: Cooperativa, Competitiva, Peer-tutoring, Isolamento
- Gruppo-Individuo: Inclusione, Esclusione, Leadership, Followership
- Famiglia-Scuola: Alleanza, Tensione, Distacco, Coinvolgimento

**DINAMICHE DI COMUNICAZIONE:**
- Modalità comunicative (assertiva, passiva, aggressiva)
- Frequenza interazioni
- Qualità del dialogo
- Ascolto reciproco
- Feedback bidirezionale

**DINAMICHE DI POTERE:**
- Autorità vs Autoritarismo
- Partecipazione decisionale
- Empowerment studenti
- Gerarchia vs Orizzontalità

**CLIMATE RELAZIONALE:**
- Fiducia reciproca
- Rispetto delle diversità
- Supporto emotivo
- Gestione conflitti
- Collaborazione vs Competizione

Per ogni dinamica relazionale:
- Qualità della relazione
- Impatto sull'apprendimento
- Fattori facilitanti/ostacolanti
- Suggerimenti miglioramento

Massimo 15 categorie relazionali.

Testo da analizzare:"""
    },
    
    "aspettative_percezioni": {
        "nome": "🎭 Analisi Aspettative e Percezioni",
        "descrizione": "Esplora aspettative, percezioni e rappresentazioni mentali",
        "prompt": """Analizza aspettative, percezioni e rappresentazioni mentali nei commenti.

Identifica:

**ASPETTATIVE:**
- Verso se stessi (autoefficacia, obiettivi personali)
- Verso insegnanti (supporto, competenza, equità)
- Verso compagni (collaborazione, rispetto)
- Verso istituzione (organizzazione, risorse)
- Verso futuro (opportunità, sbocchi)

**PERCEZIONI:**
- Del proprio ruolo (attivo, passivo, protagonista)
- Della qualità educativa (alta, media, bassa)
- Dell'ambiente scolastico (accogliente, freddo, stimolante)
- Delle proprie competenze (adeguate, insufficienti)
- Del sistema valutativo (giusto, ingiusto, trasparente)

**RAPPRESENTAZIONI MENTALI:**
- Concetto di apprendimento
- Idea di successo/fallimento
- Visione del proprio futuro
- Significato dell'educazione
- Ruolo della scuola nella società

**DISCREPANZE:**
- Aspettative vs Realtà
- Percezioni vs Fatti oggettivi
- Autorappresentazione vs Etero-percezione

Per ogni categoria:
- Livello di realismo
- Impatto motivazionale
- Fonti di influenza
- Possibilità di allineamento

Massimo 17 categorie percettive.

Testo da analizzare:"""
    }
}

print("✅ Template di prompt caricati e disponibili!")

✅ Template di prompt caricati e disponibili!


## 📁 4. Caricamento File Excel e Selezione Analisi

Carica il file Excel dalla cartella `dati` o inserisci un percorso personalizzato, poi seleziona automaticamente la colonna da analizzare e il tipo di analisi desiderato.

### 🔄 Funzionalità Automatiche:
- **📂 File dalla cartella `dati`**: Menu a tendina con tutti i file Excel disponibili
- **✏️ Percorso personalizzato**: Opzione per inserire un percorso manualmente
- **🔄 Aggiornamento dinamico**: Pulsante per ricaricare la lista file
- **🔍 Rilevamento colonne**: Il sistema rileva automaticamente tutte le colonne del file
- **📊 Conteggio commenti**: Mostra il numero di commenti non vuoti per ogni colonna
- **🎯 Selezione tipo analisi**: Menu a tendina con 9 tipi di analisi specializzate
- **🤖 Prompt automatico**: Il prompt viene caricato automaticamente in base al tipo selezionato

### 📋 Tipi di Analisi Disponibili:
1. **📊 Sentiment Analysis** - Analizza emozioni e sentimenti
2. **⚖️ Pro e Contro** - Identifica vantaggi e svantaggi  
3. **👥 Comportamenti** - Analizza comportamenti e atteggiamenti
4. **💝 Emozioni Dettagliate** - Analisi emotiva approfondita
5. **🎯 Motivazioni** - Identifica motivazioni e driver
6. **🔍 Bisogni e Necessità** - Rileva bisogni emergenti
7. **🚧 Barriere e Ostacoli** - Identifica difficoltà
8. **🤝 Relazioni e Dinamiche** - Analizza relazioni interpersonali
9. **🎭 Aspettative e Percezioni** - Esplora aspettative

### 📁 File Disponibili nella Cartella `dati`:
I file Excel presenti nella cartella `dati` vengono automaticamente rilevati e mostrati nel menu a tendina. Usa il pulsante "🔄 Aggiorna Lista" se aggiungi nuovi file.

## ⚙️ 3.1. Configurazione Parametri Avanzati

Configura i parametri per l'analisi avanzata con coefficienti di confidenza.

In [18]:
# ⚙️ CONFIGURAZIONE PARAMETRI AVANZATI

# Widget per soglia di confidenza
confidenza_slider = widgets.FloatSlider(
    value=0.3,
    min=0.1,
    max=0.9,
    step=0.1,
    description='Soglia Confidenza:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Widget per abilitare etichette multiple
multiple_labels_checkbox = widgets.Checkbox(
    value=True,
    description='Permetti etichette multiple per cella',
    style={'description_width': 'initial'}
)

# Widget per soglia etichette secondarie
secondarie_slider = widgets.FloatSlider(
    value=0.4,
    min=0.2,
    max=0.8,
    step=0.1,
    description='Soglia Secondarie:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Widget per modalità di analisi
modalita_dropdown = widgets.Dropdown(
    options=[
        ('🎯 Strict - Rigoroso nell\'etichettatura', 'strict'),
        ('⚖️ Balanced - Equilibrio qualità/copertura', 'balanced'),
        ('🌐 Loose - Massima copertura', 'loose')
    ],
    value='balanced',
    description='Modalità:',
    style={'description_width': 'initial'}
)

# Widget per numero massimo etichette dinamiche
max_etichette_slider = widgets.IntSlider(
    value=20,
    min=10,
    max=30,
    step=1,
    description='Max Etichette:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Output per anteprima configurazione
config_output = widgets.Output()

def aggiorna_anteprima_config(*args):
    """Aggiorna l'anteprima della configurazione"""
    with config_output:
        clear_output()
        print("🔧 CONFIGURAZIONE ATTUALE:")
        print(f"   🎯 Soglia confidenza principale: {confidenza_slider.value:.1f}")
        print(f"   🏷️ Etichette multiple: {'✅ Sì' if multiple_labels_checkbox.value else '❌ No'}")
        print(f"   📋 Soglia etichette secondarie: {secondarie_slider.value:.1f}")
        print(f"   ⚙️ Modalità analisi: {modalita_dropdown.value.upper()}")
        print(f"   📊 Massimo etichette dinamiche: {max_etichette_slider.value}")
        
        # Spiegazione modalità
        if modalita_dropdown.value == 'strict':
            print("\n📘 Modalità STRICT:")
            print("   • Etichettatura rigorosa con alta precisione")
            print("   • Poche etichette ma molto affidabili")
            print("   • Coefficienti alti richiesti")
        elif modalita_dropdown.value == 'balanced':
            print("\n📗 Modalità BALANCED:")
            print("   • Equilibrio tra precisione e copertura")
            print("   • Buona qualità con copertura ampia")
            print("   • Coefficienti moderati accettati")
        else:
            print("\n📙 Modalità LOOSE:")
            print("   • Massima copertura dell'etichettatura")
            print("   • Anche coefficienti bassi sono considerati")
            print("   • Più etichette per cella")

# Collega callback per aggiornamento automatico
confidenza_slider.observe(aggiorna_anteprima_config, names='value')
multiple_labels_checkbox.observe(aggiorna_anteprima_config, names='value')
secondarie_slider.observe(aggiorna_anteprima_config, names='value')
modalita_dropdown.observe(aggiorna_anteprima_config, names='value')
max_etichette_slider.observe(aggiorna_anteprima_config, names='value')

# Configurazione iniziale
aggiorna_anteprima_config()

# Pulsante per salvare configurazione
salva_config_button = widgets.Button(
    description='💾 Salva Configurazione',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def salva_configurazione(b):
    """Salva la configurazione nei parametri globali"""
    global SOGLIA_CONFIDENZA, ETICHETTE_MULTIPLE, SOGLIA_SECONDARIE, MODALITA_ANALISI, MAX_ETICHETTE_DINAMICHE
    
    SOGLIA_CONFIDENZA = confidenza_slider.value
    ETICHETTE_MULTIPLE = multiple_labels_checkbox.value
    SOGLIA_SECONDARIE = secondarie_slider.value
    MODALITA_ANALISI = modalita_dropdown.value
    MAX_ETICHETTE_DINAMICHE = max_etichette_slider.value
    
    # Adatta le soglie in base alla modalità
    if MODALITA_ANALISI == 'strict':
        SOGLIA_CONFIDENZA = max(SOGLIA_CONFIDENZA, 0.6)
        SOGLIA_SECONDARIE = max(SOGLIA_SECONDARIE, 0.5)
    elif MODALITA_ANALISI == 'loose':
        SOGLIA_CONFIDENZA = min(SOGLIA_CONFIDENZA, 0.3)
        SOGLIA_SECONDARIE = min(SOGLIA_SECONDARIE, 0.3)
    
    with config_output:
        clear_output()
        print("✅ Configurazione salvata con successo!")
        print(f"📊 Parametri applicati:")
        print(f"   🎯 Soglia confidenza: {SOGLIA_CONFIDENZA:.1f}")
        print(f"   🏷️ Etichette multiple: {ETICHETTE_MULTIPLE}")
        print(f"   📋 Soglia secondarie: {SOGLIA_SECONDARIE:.1f}")
        print(f"   ⚙️ Modalità: {MODALITA_ANALISI.upper()}")
        print(f"   📊 Max etichette: {MAX_ETICHETTE_DINAMICHE}")

salva_config_button.on_click(salva_configurazione)

# Layout interfaccia
config_interface = widgets.VBox([
    widgets.HTML("<h3>⚙️ Configurazione Parametri Avanzati</h3>"),
    widgets.HTML("<p><em>Personalizza i parametri per l'analisi con coefficienti</em></p>"),
    
    confidenza_slider,
    widgets.HTML("<small>🎯 Coefficiente minimo per considerare un'etichetta valida</small>"),
    
    secondarie_slider,
    widgets.HTML("<small>📋 Coefficiente minimo per etichette secondarie aggiuntive</small>"),
    
    multiple_labels_checkbox,
    widgets.HTML("<small>🏷️ Permette più etichette per cella se superano le soglie</small>"),
    
    modalita_dropdown,
    widgets.HTML("<small>⚙️ Strategia generale di etichettatura</small>"),
    
    max_etichette_slider,
    widgets.HTML("<small>📊 Numero massimo di etichette dinamiche da generare</small>"),
    
    salva_config_button,
    config_output
])

# Inizializza parametri globali con valori predefiniti
SOGLIA_CONFIDENZA = 0.3
ETICHETTE_MULTIPLE = True
SOGLIA_SECONDARIE = 0.4
MODALITA_ANALISI = 'balanced'
MAX_ETICHETTE_DINAMICHE = 20

display(config_interface)

In [19]:
# Funzione per scansionare i file nella cartella dati
def scansiona_file_dati():
    """Scansiona la cartella dati e restituisce i file Excel disponibili"""
    cartella_dati = "dati"
    file_excel = []
    
    try:
        if os.path.exists(cartella_dati):
            for file in os.listdir(cartella_dati):
                if file.endswith(('.xlsx', '.xls')):
                    file_path = os.path.join(cartella_dati, file)
                    # Crea una descrizione più leggibile
                    nome_breve = file.replace('.xlsx', '').replace('.xls', '')
                    if len(nome_breve) > 60:
                        nome_breve = nome_breve[:57] + "..."
                    file_excel.append((f"📊 {nome_breve}", file_path))
        
        # Aggiungi opzione per percorso personalizzato
        file_excel.append(("✏️ Inserisci percorso personalizzato...", "custom"))
        
    except Exception as e:
        print(f"⚠️ Errore nella scansione della cartella dati: {e}")
        file_excel = [("✏️ Inserisci percorso personalizzato...", "custom")]
    
    return file_excel

# Widget dropdown per selezionare il file dalla cartella dati
file_dropdown = widgets.Dropdown(
    options=scansiona_file_dati(),
    description='File disponibili:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)

# Widget per inserire il percorso personalizzato (inizialmente nascosto)
file_path_custom = widgets.Text(
    value='',
    placeholder='Inserisci il percorso completo del file Excel...',
    description='Percorso custom:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%', display='none')
)

# Widget dropdown per selezionare la colonna (sarà popolato automaticamente)
colonna_dropdown = widgets.Dropdown(
    options=[],
    description='Colonna commenti:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
    disabled=True
)

# Widget dropdown per il tipo di analisi
tipo_analisi_options = [
    ('📊 Sentiment Analysis - Analizza emozioni e sentimenti', 'sentiment_analysis'),
    ('⚖️ Pro e Contro - Identifica vantaggi e svantaggi', 'pro_contro'),
    ('👥 Comportamenti - Analizza comportamenti e atteggiamenti', 'comportamenti'),
    ('💝 Emozioni Dettagliate - Analisi emotiva approfondita', 'emozioni_dettagliate'),
    ('🎯 Motivazioni - Identifica motivazioni e driver', 'motivazioni'),
    ('🔍 Bisogni e Necessità - Rileva bisogni emergenti', 'bisogni_necessita'),
    ('🚧 Barriere e Ostacoli - Identifica difficoltà', 'barriere_ostacoli'),
    ('🤝 Relazioni e Dinamiche - Analizza relazioni interpersonali', 'relazioni_dinamiche'),
    ('🎭 Aspettative e Percezioni - Esplora aspettative', 'aspettative_percezioni')
]

tipo_analisi_dropdown = widgets.Dropdown(
    options=tipo_analisi_options,
    description='Tipo analisi:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Pulsante per caricare il file
load_button = widgets.Button(
    description='📂 Carica File',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

# Pulsante per ricaricare i file dalla cartella dati
refresh_button = widgets.Button(
    description='🔄 Aggiorna Lista',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

# Pulsante per confermare la selezione
confirm_selection_button = widgets.Button(
    description='✅ Conferma Selezione',
    button_style='success',
    layout=widgets.Layout(width='200px'),
    disabled=True
)

# Output area per mostrare i risultati
output_area = widgets.Output()

def on_file_dropdown_change(change):
    """Callback quando cambia la selezione del file"""
    if change['new'] == 'custom':
        file_path_custom.layout.display = 'block'
    else:
        file_path_custom.layout.display = 'none'
        file_path_custom.value = ''

def on_refresh_button_clicked(b):
    """Ricarica la lista dei file dalla cartella dati"""
    with output_area:
        print("🔄 Aggiornamento lista file...")
    
    try:
        nuovi_file = scansiona_file_dati()
        file_dropdown.options = nuovi_file
        
        with output_area:
            print(f"✅ Lista aggiornata! Trovati {len(nuovi_file)-1} file Excel nella cartella dati")
    except Exception as e:
        with output_area:
            print(f"❌ Errore nell'aggiornamento: {e}")

def on_load_button_clicked(b):
    with output_area:
        clear_output()
        try:
            # Determina il percorso del file
            if file_dropdown.value == 'custom':
                file_path = file_path_custom.value.strip()
                if not file_path:
                    print("❌ Inserisci il percorso del file Excel!")
                    return
            else:
                file_path = file_dropdown.value
            
            # Carica il file Excel
            global df_temp
            df_temp = pd.read_excel(file_path)
            
            print(f"✅ File caricato con successo!")
            print(f"📁 Percorso: {file_path}")
            print(f"📊 Dimensioni: {df_temp.shape[0]} righe, {df_temp.shape[1]} colonne")
            
            # Popola automaticamente il dropdown delle colonne
            colonne_disponibili = [(f"{col} ({df_temp[col].notna().sum()} commenti)", col) for col in df_temp.columns]
            colonna_dropdown.options = colonne_disponibili
            colonna_dropdown.disabled = False
            confirm_selection_button.disabled = False
            
            print(f"📋 Colonne rilevate automaticamente:")
            for i, col in enumerate(df_temp.columns, 1):
                commenti_validi = df_temp[col].notna().sum()
                tipo_dati = str(df_temp[col].dtype)
                print(f"   {i}. {col}: {commenti_validi} commenti non vuoti ({tipo_dati})")
            
            print(f"\n💡 Seleziona la colonna contenente i commenti da analizzare")
            print(f"💡 Scegli il tipo di analisi dal menu a tendina")
            print(f"💡 Clicca 'Conferma Selezione' per procedere")
                
        except Exception as e:
            print(f"❌ Errore nel caricamento del file: {str(e)}")
            print(f"💡 Verifica che il percorso sia corretto e che il file sia un Excel valido")

def on_confirm_selection(b):
    with output_area:
        print("\n" + "="*50)
        try:
            if colonna_dropdown.value is None:
                print("❌ Seleziona una colonna!")
                return
            
            global df, colonna_riferimento, tipo_analisi, tipo_analisi_nome, file_analizzato
            
            # Assegna le variabili globali
            df = df_temp.copy()
            colonna_riferimento = colonna_dropdown.value
            tipo_analisi = tipo_analisi_dropdown.value
            
            # Salva info del file per riferimento
            if file_dropdown.value == 'custom':
                file_analizzato = file_path_custom.value.strip()
            else:
                file_analizzato = file_dropdown.value
            
            # Trova il nome leggibile del tipo di analisi
            tipo_analisi_nome = next(name for name, value in tipo_analisi_options if value == tipo_analisi)
            
            print(f"✅ CONFIGURAZIONE CONFERMATA!")
            print(f"📁 File: {file_analizzato}")
            print(f"📋 Colonna selezionata: {colonna_riferimento}")
            print(f"🎯 Tipo analisi: {tipo_analisi_nome}")
            print(f"📝 Commenti da analizzare: {df[colonna_riferimento].notna().sum()}")
            print(f"❌ Commenti vuoti: {df[colonna_riferimento].isna().sum()}")
            
            # Mostra anteprima
            print(f"\n🔍 Anteprima primi 3 commenti dalla colonna '{colonna_riferimento}':")
            for i, comment in enumerate(df[colonna_riferimento].dropna().head(3)):
                print(f"{i+1}. {str(comment)[:100]}...")
                
            print(f"\n🚀 Ora puoi procedere alla sezione successiva per avviare l'analisi!")
            
            # Aggiorna automaticamente il prompt per il tipo di analisi selezionato
            aggiorna_prompt_automatico()
                
        except Exception as e:
            print(f"❌ Errore nella conferma: {str(e)}")

# Collega i callback
file_dropdown.observe(on_file_dropdown_change, names='value')
load_button.on_click(on_load_button_clicked)
refresh_button.on_click(on_refresh_button_clicked)
confirm_selection_button.on_click(on_confirm_selection)

# Mostra i widget
display(widgets.VBox([
    widgets.HTML("<h3>📁 Caricamento e Configurazione File</h3>"),
    widgets.HTML("<p><em>1. Seleziona un file dalla cartella 'dati' o inserisci un percorso personalizzato</em></p>"),
    widgets.HBox([file_dropdown, refresh_button]),
    file_path_custom,
    load_button,
    widgets.HTML("<p><em>2. Seleziona la colonna contenente i commenti</em></p>"),
    colonna_dropdown,
    widgets.HTML("<p><em>3. Scegli il tipo di analisi da eseguire</em></p>"),
    tipo_analisi_dropdown,
    confirm_selection_button,
    output_area
]))

## ✏️ 5. Prompt Automatico

Il prompt di analisi viene caricato automaticamente in base al tipo di analisi selezionato nella sezione precedente.

In [21]:
# Il prompt viene ora selezionato automaticamente in base al tipo di analisi scelto
# Questo widget mostra il prompt che verrà utilizzato

prompt_display = widgets.HTML(
    value="<p><em>Il prompt verrà caricato automaticamente quando confermi la selezione del file e tipo di analisi.</em></p>",
    layout=widgets.Layout(width='90%')
)

# Widget per visualizzare il prompt corrente (read-only)
prompt_preview = widgets.Textarea(
    value='',
    description='Prompt automatico:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%', height='200px'),
    disabled=True
)

# Pulsante per confermare il prompt (opzionale, per visualizzazione)
confirm_prompt_button = widgets.Button(
    description='✅ Prompt Confermato',
    button_style='success',
    layout=widgets.Layout(width='200px'),
    disabled=True
)

# Output area per il prompt
prompt_output = widgets.Output()

def aggiorna_prompt_automatico():
    """Aggiorna il prompt in base al tipo di analisi selezionato"""
    if 'tipo_analisi' in globals() and tipo_analisi in template_prompts:
        global prompt_personalizzato
        prompt_personalizzato = template_prompts[tipo_analisi]['prompt']
        
        # Aggiorna la visualizzazione
        prompt_preview.value = prompt_personalizzato
        prompt_display.value = f"""
        <div style='background: #e8f5e8; padding: 10px; border-radius: 5px; margin: 10px 0;'>
        <h4>✅ Prompt Automatico Caricato</h4>
        <p><b>Tipo di analisi:</b> {template_prompts[tipo_analisi]['nome']}</p>
        <p><b>Descrizione:</b> {template_prompts[tipo_analisi]['descrizione']}</p>
        <p><b>Lunghezza prompt:</b> {len(prompt_personalizzato)} caratteri</p>
        </div>
        """
        confirm_prompt_button.disabled = False
        
        with prompt_output:
            clear_output()
            print("✅ Prompt automatico caricato e confermato!")
            print(f"🎯 Tipo: {template_prompts[tipo_analisi]['nome']}")
            print(f"📝 Lunghezza: {len(prompt_personalizzato)} caratteri")
            print("🚀 Puoi procedere all'analisi!")
    else:
        prompt_display.value = "<p><em>⚠️ Seleziona prima il tipo di analisi nella sezione precedente.</em></p>"
        prompt_preview.value = ""
        confirm_prompt_button.disabled = True

def on_confirm_prompt(b):
    with prompt_output:
        clear_output()
        print("✅ Prompt confermato e pronto per l'analisi!")

confirm_prompt_button.on_click(on_confirm_prompt)

# Controlla se il tipo di analisi è già stato selezionato
if 'tipo_analisi' in globals():
    aggiorna_prompt_automatico()

# Mostra i widget
display(widgets.VBox([
    widgets.HTML("<h3>✏️ Prompt Automatico</h3>"),
    widgets.HTML("<p><em>Il prompt viene selezionato automaticamente in base al tipo di analisi scelto</em></p>"),
    prompt_display,
    prompt_preview,
    confirm_prompt_button,
    prompt_output
]))

## 🚀 6. Esecuzione Analisi Completa

Avvia l'analisi completa dei commenti utilizzando il prompt personalizzato.

In [ ]:
# Pulsante per avviare l'analisi avanzata
start_analysis_button = widgets.Button(
    description='🚀 Avvia Analisi Avanzata',
    button_style='warning',
    layout=widgets.Layout(width='220px')
)

# Progress bar avanzata
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progresso:',
    bar_style='info',
    style={'bar_color': '#1f77b4'},
    layout=widgets.Layout(width='80%')
)

# Label per fase corrente
fase_label = widgets.HTML(value="<b>⏳ In attesa di avvio...</b>")

# Output area per l'analisi
analysis_output = widgets.Output()

def on_start_analysis_avanzata(b):
    global etichette_dinamiche_globali, risultati_etichettatura_globali
    
    with analysis_output:
        clear_output()
        
        try:
            # Verifica configurazione
            if 'df' not in globals() or 'prompt_personalizzato' not in globals():
                print("❌ Assicurati di aver caricato il file e confermato il prompt!")
                return
            
            if 'SOGLIA_CONFIDENZA' not in globals():
                print("❌ Configura prima i parametri avanzati!")
                return
            
            print("🚀 AVVIO ANALISI AVANZATA CON COEFFICIENTI")
            print("=" * 60)
            
            # === FASE 1: ANALISI GLOBALE ===
            fase_label.value = "<b>🔍 FASE 1: Analisi globale della colonna...</b>"
            progress_bar.value = 5
            print("🔍 FASE 1: Analisi globale per generare etichette dinamiche")
            
            output_dir = create_output_dir(tipo_analisi)
            print(f"📁 Cartella output: {output_dir}")
            
            progress_bar.value = 15
            print(f"📊 Configurazione: Soglia {SOGLIA_CONFIDENZA:.1f} | Modalità {MODALITA_ANALISI.upper()}")
            print(f"📝 Commenti da analizzare: {df[colonna_riferimento].notna().sum()}")
            
            # Analisi globale della colonna
            fase_label.value = "<b>🤖 Generazione etichette dinamiche...</b>"
            risultati_globali = analizza_colonna_completa(df, colonna_riferimento, prompt_personalizzato, tipo_analisi)
            progress_bar.value = 35
            
            etichette_dinamiche_globali = risultati_globali["etichette_dinamiche"]
            print(f"✅ FASE 1 COMPLETATA: {len(etichette_dinamiche_globali)} etichette dinamiche generate")
            
            # Mostra etichette generate
            print("\\n🏷️ ETICHETTE DINAMICHE GENERATE:")
            for i, (nome, info) in enumerate(etichette_dinamiche_globali.items(), 1):
                print(f"{i:2d}. {nome}: {info['descrizione'][:60]}...")
            
            # Salva analisi globale
            analisi_path = os.path.join(output_dir, f'analisi_globale_{tipo_analisi}.txt')
            with open(analisi_path, 'w', encoding='utf-8') as f:
                f.write(risultati_globali["analisi_completa"])
            print(f"💾 Analisi globale salvata: {analisi_path}")
            
            # === FASE 2: ETICHETTATURA CON COEFFICIENTI ===
            fase_label.value = "<b>🏷️ FASE 2: Preparazione etichettatura...</b>"
            progress_bar.value = 40
            print("\\n🏷️ FASE 2: Etichettatura avanzata con coefficienti di confidenza")
            
            # Conteggio commenti da analizzare
            commenti_validi = df[colonna_riferimento].notna().sum()
            print(f"📝 Commenti da etichettare: {commenti_validi}")
            print(f"🎯 Etichette dinamiche generate: {len(etichette_dinamiche_globali)}")
            print(f"⚙️ Soglia confidenza: {SOGLIA_CONFIDENZA:.1f}")
            print("🚀 Inizio etichettatura con indicatori di progresso dettagliati...")
            
            # Etichettatura con coefficienti
            risultati_etichettatura_globali = etichetta_con_coefficiente(
                df, 
                etichette_dinamiche_globali, 
                colonna_riferimento, 
                tipo_analisi, 
                SOGLIA_CONFIDENZA
            )
            progress_bar.value = 80
            
            print(f"✅ FASE 2 COMPLETATA: Tutti i {df[colonna_riferimento].notna().sum()} commenti etichettati con successo!")
            print(f"🎯 Etichettatura al 100% completata con coefficienti di confidenza")
            
            # Mostra statistiche confidenza
            stats = risultati_etichettatura_globali["statistiche_confidenza"]
            print(f"\\n📊 STATISTICHE CONFIDENZA:")
            print(f"   📈 Media coefficienti: {stats['media_coefficienti']:.3f}")
            print(f"   🟢 Alta confidenza (≥0.7): {stats['alta_confidenza']} celle")
            print(f"   🟡 Media confidenza (0.4-0.7): {stats['media_confidenza']} celle")
            print(f"   🟠 Bassa confidenza (0.1-0.4): {stats['bassa_confidenza']} celle")
            print(f"   🔴 Molto bassa (<0.1): {stats['molto_bassa']} celle")
            
            # === FASE 3: SALVATAGGIO AVANZATO ===
            fase_label.value = "<b>💾 FASE 3: Salvataggio risultati avanzati...</b>"
            progress_bar.value = 90
            print("\\n💾 FASE 3: Salvataggio risultati avanzati")
            
            # Salva risultati completi
            excel_path, etichette_path, stats_path = salva_risultati_avanzati(
                df, 
                risultati_etichettatura_globali, 
                etichette_dinamiche_globali, 
                output_dir, 
                tipo_analisi
            )
            
            progress_bar.value = 100
            fase_label.value = "<b>✅ ANALISI AVANZATA COMPLETATA!</b>"
            
            print("\\n🎉 ANALISI AVANZATA COMPLETATA CON SUCCESSO!")
            print("=" * 60)
            print(f"📊 Report Excel avanzato: {excel_path}")
            print(f"🏷️ Etichette dinamiche: {etichette_path}")
            print(f"📈 Statistiche confidenza: {stats_path}")
            
            # Anteprima risultati
            print("\\n🔍 ANTEPRIMA RISULTATI AVANZATI:")
            preview_df = df[['id', colonna_riferimento]].copy()
            preview_df['Etichetta_Principale'] = risultati_etichettatura_globali["etichette_principali"]
            preview_df['Coefficiente'] = [f"{c:.2f}" for c in risultati_etichettatura_globali["coefficienti_principali"]]
            preview_df['Confidenza'] = [f"{c:.2f}" for c in risultati_etichettatura_globali["confidenza_media"]]
            
            print(preview_df.head(5).to_string(index=False, max_colwidth=30))
            
        except Exception as e:
            print(f"❌ ERRORE DURANTE L'ANALISI AVANZATA: {str(e)}")
            fase_label.value = f"<b style='color: red;'>❌ Errore: {str(e)[:50]}...</b>"
            progress_bar.value = 0
            import traceback
            print(f"\\n🔍 Dettagli errore:\\n{traceback.format_exc()}")

start_analysis_button.on_click(on_start_analysis_avanzata)

# Layout interfaccia esecuzione
execution_interface = widgets.VBox([
    widgets.HTML("<h3>🚀 Esecuzione Analisi Avanzata</h3>"),
    widgets.HTML("<p><em>Sistema a 2 fasi: analisi globale + etichettatura con coefficienti</em></p>"),
    start_analysis_button,
    fase_label,
    progress_bar,
    analysis_output
])

display(execution_interface)

## 🧪 Test Sistema di Progresso

Il nuovo sistema di progresso mostra percentuale e conteggio dettagliato durante l'analisi avanzata.

In [1]:
# 🧪 Test del nuovo sistema di progresso dettagliato

print("🧪 SIMULAZIONE PROGRESSO ANALISI AVANZATA")
print("=" * 50)
print()

# Simula alcuni valori di test
test_cases = [
    (1, 100),   # Primo commento
    (4, 100),   # Quarto commento  
    (25, 100),  # Quarto di analisi completato
    (50, 100),  # Metà analisi
    (75, 100),  # Tre quarti
    (99, 100),  # Quasi finito
    (100, 100), # Completato
    (3, 20),    # File piccolo
    (15, 50),   # File medio
]

print("📊 Esempi di indicatori di progresso durante l'analisi:")
print()

for numero_corrente, totale_commenti in test_cases:
    percentuale = int((numero_corrente / totale_commenti) * 100)
    progress_fase2 = 40 + int((numero_corrente / totale_commenti) * 40)
    
    # Simula il formato che apparirà nel widget fase_label
    fase_display = f"🏷️ FASE 2: Etichettatura {percentuale}% {numero_corrente}/{totale_commenti}"
    
    print(f"├─ {fase_display}")
    print(f"│  Progress Bar: {progress_fase2}%")
    print(f"│  Log: Analizzando commento {numero_corrente}/{totale_commenti} ({percentuale}%)")
    
    if numero_corrente % 10 == 0 or numero_corrente == totale_commenti:
        print(f"│  ✅ Milestone: {numero_corrente} commenti elaborati")
    
    print("│")

print()
print("✅ Il formato sarà esattamente: **p% numero_corrente/totale**")
print("📊 La progress bar andrà da 40% a 80% durante la Fase 2")
print("🔄 Aggiornamento in tempo reale per ogni commento analizzato")

🧪 SIMULAZIONE PROGRESSO ANALISI AVANZATA

📊 Esempi di indicatori di progresso durante l'analisi:

├─ 🏷️ FASE 2: Etichettatura 1% 1/100
│  Progress Bar: 40%
│  Log: Analizzando commento 1/100 (1%)
│
├─ 🏷️ FASE 2: Etichettatura 4% 4/100
│  Progress Bar: 41%
│  Log: Analizzando commento 4/100 (4%)
│
├─ 🏷️ FASE 2: Etichettatura 25% 25/100
│  Progress Bar: 50%
│  Log: Analizzando commento 25/100 (25%)
│
├─ 🏷️ FASE 2: Etichettatura 50% 50/100
│  Progress Bar: 60%
│  Log: Analizzando commento 50/100 (50%)
│  ✅ Milestone: 50 commenti elaborati
│
├─ 🏷️ FASE 2: Etichettatura 75% 75/100
│  Progress Bar: 70%
│  Log: Analizzando commento 75/100 (75%)
│
├─ 🏷️ FASE 2: Etichettatura 99% 99/100
│  Progress Bar: 79%
│  Log: Analizzando commento 99/100 (99%)
│
├─ 🏷️ FASE 2: Etichettatura 100% 100/100
│  Progress Bar: 80%
│  Log: Analizzando commento 100/100 (100%)
│  ✅ Milestone: 100 commenti elaborati
│
├─ 🏷️ FASE 2: Etichettatura 15% 3/20
│  Progress Bar: 46%
│  Log: Analizzando commento 3/20 (15%)
│
├

### 🎯 Migliorie Implementate

Il sistema di progresso è stato migliorato con le seguenti funzionalità:

#### 📊 **Indicatore Dettagliato**
- **Formato**: `p% numero_corrente/totale` (es: `25% 25/100`)
- **Aggiornamento**: In tempo reale per ogni commento analizzato
- **Posizione**: Widget `fase_label` nell'interfaccia

#### 📈 **Progress Bar Sincronizzata**
- **Range Fase 2**: 40% → 80% (40% riservato alla fase di etichettatura)
- **Calcolo**: `40 + (progresso_corrente / totale) * 40`
- **Fluidità**: Aggiornamento granulare ad ogni iterazione

#### 📝 **Logging Migliorato**
- **Console**: Mostra progresso ogni commento con percentuale
- **Milestone**: Messaggi speciali ogni 10 commenti o fine blocco
- **Dettagli**: Include numero corrente, totale e percentuale

#### 🔄 **Esperienza Utente**
- **Visibilità**: L'utente vede sempre dove si trova nell'analisi
- **Prevedibilità**: Può stimare il tempo rimanente
- **Feedback**: Conferma continua che il processo sta procedendo

Questo sistema fornisce un feedback molto più dettagliato durante l'analisi avanzata, permettendo all'utente di monitorare precisamente il progresso dell'elaborazione.

## 📊 7. Visualizzazione Risultati

Esplora i risultati dell'analisi e le statistiche generate.

In [ ]:
# 📊 Visualizzazione Avanzata dei Risultati
def visualizza_risultati_avanzati():
    """Visualizza risultati avanzati con metriche di qualità e grafici"""
    
    if 'risultati_etichettatura_globali' not in globals() or 'etichette_dinamiche_globali' not in globals():
        print("❌ Esegui prima l'analisi avanzata per vedere i risultati!")
        return
    
    print("📊 RIEPILOGO ANALISI AVANZATA")
    print("=" * 60)
    print(f"📁 Tipo di analisi: {tipo_analisi}")
    print(f"📄 File analizzato: {file_path_widget.value}")
    print(f"📋 Colonna analizzata: {colonna_riferimento}")
    print(f"📝 Numero totale commenti: {len(df)}")
    print(f"✅ Commenti analizzati: {df[colonna_riferimento].notna().sum()}")
    print(f"❌ Commenti vuoti: {df[colonna_riferimento].isna().sum()}")
    
    # Statistiche etichette dinamiche
    print(f"\\n🏷️ ETICHETTE DINAMICHE GENERATE: {len(etichette_dinamiche_globali)}")
    print("-" * 40)
    for i, (nome, info) in enumerate(etichette_dinamiche_globali.items(), 1):
        print(f"{i:2d}. {nome}")
        print(f"    📝 {info['descrizione'][:80]}...")
        if info['esempi']:
            print(f"    💡 Esempi: {info['esempi'][:60]}...")
        print()
    
    # Statistiche di qualità
    stats = risultati_etichettatura_globali["statistiche_confidenza"]
    print("📈 METRICHE DI QUALITÀ")
    print("-" * 30)
    print(f"🎯 Media coefficienti: {stats['media_coefficienti']:.3f}")
    print(f"🟢 Alta confidenza (≥0.7): {stats['alta_confidenza']} celle ({stats['alta_confidenza']/len(df)*100:.1f}%)")
    print(f"🟡 Media confidenza (0.4-0.7): {stats['media_confidenza']} celle ({stats['media_confidenza']/len(df)*100:.1f}%)")
    print(f"🟠 Bassa confidenza (0.1-0.4): {stats['bassa_confidenza']} celle ({stats['bassa_confidenza']/len(df)*100:.1f}%)")
    print(f"🔴 Molto bassa (<0.1): {stats['molto_bassa']} celle ({stats['molto_bassa']/len(df)*100:.1f}%)")
    
    # Distribuzione etichette principali
    print("\\n🏆 TOP 10 ETICHETTE PRINCIPALI")
    print("-" * 35)
    etichette_principali = risultati_etichettatura_globali["etichette_principali"]
    from collections import Counter
    conteggi = Counter([e for e in etichette_principali if e not in ["Vuota", "Errore", "Incerto"]])
    
    for i, (etichetta, count) in enumerate(conteggi.most_common(10), 1):
        # Calcola coefficiente medio per questa etichetta
        coefficienti_etichetta = [
            risultati_etichettatura_globali["coefficienti_principali"][j] 
            for j, e in enumerate(etichette_principali) 
            if e == etichetta
        ]
        coeff_medio = sum(coefficienti_etichetta) / len(coefficienti_etichetta) if coefficienti_etichetta else 0
        print(f"{i:2d}. {etichetta}: {count} occorrenze (coeff. medio: {coeff_medio:.2f})")
    
    # Anteprima dati avanzati
    print("\\n📋 ANTEPRIMA DATI AVANZATI")
    print("-" * 25)
    preview_data = []
    for i in range(min(8, len(df))):
        commento = str(df.iloc[i][colonna_riferimento])[:40] + "..." if len(str(df.iloc[i][colonna_riferimento])) > 40 else str(df.iloc[i][colonna_riferimento])
        etichetta = risultati_etichettatura_globali["etichette_principali"][i]
        coefficiente = risultati_etichettatura_globali["coefficienti_principali"][i]
        confidenza = risultati_etichettatura_globali["confidenza_media"][i]
        secondarie = risultati_etichettatura_globali["etichette_secondarie"][i]
        
        # Indicatore qualità
        if coefficiente >= 0.7:
            qualita = "🟢"
        elif coefficiente >= 0.4:
            qualita = "🟡"
        elif coefficiente >= 0.1:
            qualita = "🟠"
        else:
            qualita = "🔴"
        
        print(f"{i+1:2d}. {qualita} {etichetta} ({coefficiente:.2f})")
        print(f"     💬 {commento}")
        if secondarie and secondarie.strip():
            print(f"     📋 Secondarie: {secondarie}")
        print()
    
    # Grafico distribuzione qualità (semplificato)
    print("📊 DISTRIBUZIONE QUALITÀ")
    print("-" * 25)
    
    # Istogramma ASCII semplice
    totale = stats['alta_confidenza'] + stats['media_confidenza'] + stats['bassa_confidenza'] + stats['molto_bassa']
    if totale > 0:
        alta_perc = stats['alta_confidenza'] / totale
        media_perc = stats['media_confidenza'] / totale  
        bassa_perc = stats['bassa_confidenza'] / totale
        molto_bassa_perc = stats['molto_bassa'] / totale
        
        larghezza = 40
        print(f"🟢 Alta:       {'█' * int(alta_perc * larghezza):<{larghezza}} {alta_perc*100:.1f}%")
        print(f"🟡 Media:      {'█' * int(media_perc * larghezza):<{larghezza}} {media_perc*100:.1f}%")
        print(f"🟠 Bassa:      {'█' * int(bassa_perc * larghezza):<{larghezza}} {bassa_perc*100:.1f}%")
        print(f"🔴 Molto bassa: {'█' * int(molto_bassa_perc * larghezza):<{larghezza}} {molto_bassa_perc*100:.1f}%")
    
    # File generati
    print("\\n💾 FILE GENERATI")
    print("-" * 15)
    if 'output_dir' in globals():
        print(f"📁 Cartella output: {output_dir}")
        print(f"📊 Report Excel avanzato: report_avanzato_{tipo_analisi}.xlsx")
        print(f"🏷️ Etichette dinamiche: etichette_dinamiche_{tipo_analisi}.json")
        print(f"📈 Statistiche confidenza: statistiche_confidenza_{tipo_analisi}.json")
        print(f"🔍 Analisi globale: analisi_globale_{tipo_analisi}.txt")

# Widget per visualizzazione avanzata
show_advanced_results_button = widgets.Button(
    description='📊 Mostra Risultati Avanzati',
    button_style='info',
    layout=widgets.Layout(width='250px')
)

# Widget per esportare metriche
export_metrics_button = widgets.Button(
    description='📈 Esporta Metriche',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

# Widget per confronto qualità
quality_analysis_button = widgets.Button(
    description='🔍 Analisi Qualità',
    button_style='warning',
    layout=widgets.Layout(width='200px')
)

results_output = widgets.Output()

def on_show_advanced_results(b):
    with results_output:
        clear_output()
        visualizza_risultati_avanzati()

def on_export_metrics(b):
    with results_output:
        if 'risultati_etichettatura_globali' not in globals():
            print("❌ Nessun risultato da esportare!")
            return
        
        try:
            import json
            metrics_data = {
                "timestamp": datetime.now().isoformat(),
                "configurazione": {
                    "soglia_confidenza": SOGLIA_CONFIDENZA,
                    "modalita_analisi": MODALITA_ANALISI,
                    "etichette_multiple": ETICHETTE_MULTIPLE
                },
                "statistiche": risultati_etichettatura_globali["statistiche_confidenza"],
                "etichette_generate": len(etichette_dinamiche_globali),
                "commenti_analizzati": df[colonna_riferimento].notna().sum()
            }
            
            metrics_path = os.path.join(output_dir, f'metriche_dettagliate_{tipo_analisi}.json')
            with open(metrics_path, 'w', encoding='utf-8') as f:
                json.dump(metrics_data, f, ensure_ascii=False, indent=2)
            
            print(f"✅ Metriche esportate in: {metrics_path}")
        except Exception as e:
            print(f"❌ Errore nell'esportazione: {e}")

def on_quality_analysis(b):
    with results_output:
        if 'risultati_etichettatura_globali' not in globals():
            print("❌ Nessun risultato da analizzare!")
            return
        
        clear_output()
        print("🔍 ANALISI DETTAGLIATA DELLA QUALITÀ")
        print("=" * 50)
        
        coefficienti = risultati_etichettatura_globali["coefficienti_principali"]
        coefficienti_validi = [c for c in coefficienti if c > 0]
        
        if coefficienti_validi:
            print(f"📊 Statistiche coefficienti:")
            print(f"   📈 Massimo: {max(coefficienti_validi):.3f}")
            print(f"   📉 Minimo: {min(coefficienti_validi):.3f}")
            print(f"   📊 Media: {sum(coefficienti_validi)/len(coefficienti_validi):.3f}")
            print(f"   📏 Mediana: {sorted(coefficienti_validi)[len(coefficienti_validi)//2]:.3f}")
            
            # Identifica celle problematiche
            problematiche = [(i, c) for i, c in enumerate(coefficienti) if 0 < c < 0.2]
            if problematiche:
                print(f"\\n⚠️  CELLE PROBLEMATICHE (coefficiente < 0.2): {len(problematiche)}")
                for i, (idx, coeff) in enumerate(problematiche[:5]):
                    commento = str(df.iloc[idx][colonna_riferimento])[:60] + "..." if len(str(df.iloc[idx][colonna_riferimento])) > 60 else str(df.iloc[idx][colonna_riferimento])
                    etichetta = risultati_etichettatura_globali["etichette_principali"][idx]
                    print(f"   {i+1}. Riga {idx+1}: {etichetta} ({coeff:.3f})")
                    print(f"      💬 {commento}")
            
            # Celle di alta qualità
            eccellenti = [(i, c) for i, c in enumerate(coefficienti) if c >= 0.8]
            if eccellenti:
                print(f"\\n🌟 CELLE ECCELLENTI (coefficiente ≥ 0.8): {len(eccellenti)}")
                print("   Esempi di etichettatura di alta qualità:")
                for i, (idx, coeff) in enumerate(eccellenti[:3]):
                    etichetta = risultati_etichettatura_globali["etichette_principali"][idx]
                    print(f"   ⭐ {etichetta} ({coeff:.3f})")

show_advanced_results_button.on_click(on_show_advanced_results)
export_metrics_button.on_click(on_export_metrics)
quality_analysis_button.on_click(on_quality_analysis)

# Layout interfaccia risultati avanzati
advanced_results_interface = widgets.VBox([
    widgets.HTML("<h3>📊 Visualizzazione Risultati Avanzati</h3>"),
    widgets.HTML("<p><em>Esplora risultati, metriche di qualità e coefficienti di confidenza</em></p>"),
    widgets.HBox([show_advanced_results_button, export_metrics_button, quality_analysis_button]),
    results_output
])

display(advanced_results_interface)

## 🎉 Analisi Avanzata Completata!

Congratulazioni! Hai completato l'analisi interattiva avanzata dei tuoi commenti Excel utilizzando il **sistema a 2 fasi con coefficienti di confidenza**.

### 🚀 **Nuovo Sistema Avanzato:**
- **🔍 Fase 1**: Analisi globale della colonna → Etichette dinamiche generate dal contenuto reale
- **🏷️ Fase 2**: Etichettatura con coefficienti di confidenza (0.0-1.0) per ogni cella
- **📊 Metriche di qualità**: Statistiche dettagliate sulla qualità dell'etichettatura

### 📁 **File Generati Avanzati:**
- **📊 Report Excel Avanzato**: Contiene etichette principali, coefficienti, confidenza generale e etichette secondarie
- **🏷️ Etichette Dinamiche JSON**: Dizionario completo delle etichette generate automaticamente
- **📈 Statistiche Confidenza**: Metriche dettagliate sulla qualità dell'analisi
- **🔍 Analisi Globale**: Testo completo dell'analisi globale della colonna

### ⚙️ **Parametri Configurabili:**
- **🎯 Soglia Confidenza**: Coefficiente minimo per etichette valide
- **📋 Soglia Secondarie**: Limite per etichette aggiuntive
- **🏷️ Etichette Multiple**: Permetti più etichette per cella
- **⚙️ Modalità Analisi**: Strict/Balanced/Loose

### 📊 **Colonne Output Excel:**
- `Etichetta_Principale` → Etichetta con coefficiente più alto
- `Coefficiente_Principale` → Valore di confidenza (0.0-1.0)
- `Etichette_Secondarie` → Altre etichette significative
- `Confidenza_Generale` → Media ponderata di tutte le etichette
- `Coefficienti_Completi` → JSON con tutti i coefficienti

### 🔄 **Per Nuove Analisi Avanzate:**
1. **Cella 4**: Carica nuovo file Excel
2. **Cella 3.1**: Configura parametri avanzati (soglie, modalità)
3. **Cella 5**: Seleziona template di analisi o personalizza prompt
4. **Cella 6**: Avvia analisi avanzata a 2 fasi
5. **Cella 7**: Visualizza risultati e metriche di qualità

### 💡 **Vantaggi del Sistema Avanzato:**
- ✅ **Etichette dinamiche** generate dal contenuto reale
- ✅ **Coefficienti di confidenza** per ogni etichettatura
- ✅ **Metriche di qualità** dettagliate
- ✅ **Configurazione flessibile** dei parametri
- ✅ **Identificazione celle problematiche** (bassa confidenza)
- ✅ **Etichette multiple** con pesi diversi
- ✅ **Tracciabilità completa** di ogni decisione AI

### 🔍 **Interpretazione Coefficienti:**
- **🟢 0.7-1.0**: Alta confidenza - Etichettatura molto affidabile
- **🟡 0.4-0.7**: Media confidenza - Buona qualità, controllare se necessario
- **🟠 0.1-0.4**: Bassa confidenza - Rivedere manualmente
- **🔴 <0.1**: Molto bassa - Probabile errore o commento ambiguo

### 📈 **Suggerimenti per Ottimizzare:**
- **Modalità Strict**: Per analisi di alta precisione con poche etichette incerte
- **Modalità Balanced**: Equilibrio ideale per la maggior parte dei casi
- **Modalità Loose**: Per massima copertura, accettando più incertezza
- **Soglie personalizzate**: Adatta in base al tipo di dati e requisiti di qualità

---
*Sistema Avanzato di Analisi AI - Versione 2.0 con Coefficienti di Confidenza*